In [1]:
'''
Copyright (c) 2019, Crystal Xue
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
3. All advertising materials mentioning features or use of this software
   must display the following acknowledgement:
   This product includes software developed by the <organization>.
4. Neither the name of the <organization> nor the
   names of its contributors may be used to endorse or promote products
   derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY CRYSTAL XUE ''AS IS'' AND ANY
EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL CRYSTAL XUE BE LIABLE FOR ANY
DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
'''

"\nCopyright (c) 2019, Crystal Xue\nAll rights reserved.\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are met:\n1. Redistributions of source code must retain the above copyright\n   notice, this list of conditions and the following disclaimer.\n2. Redistributions in binary form must reproduce the above copyright\n   notice, this list of conditions and the following disclaimer in the\n   documentation and/or other materials provided with the distribution.\n3. All advertising materials mentioning features or use of this software\n   must display the following acknowledgement:\n   This product includes software developed by the <organization>.\n4. Neither the name of the <organization> nor the\n   names of its contributors may be used to endorse or promote products\n   derived from this software without specific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY CRYSTAL XUE ''AS IS'' AND AN

In [2]:
# This cell downloads necessary packages, which contain functions that the program will call later to scrape necessary information.
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import numpy as np

In [3]:
def get_headers():
    # Creating headers.
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    return headers

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, headers=get_headers(), stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
def get_address_price(html, isDebug=True):
    # Scrapes desired information from html code. Can be modified to include more/less information as long as content is in html file.
    addressPriceList = list()
    na = 'n/a'
    i = 0
    for li in html.select('li[class="component_property-card js-component_property-card "]'):
        # Selects each individual property
        i = i+1
        
        # Collects street address
        streetAddress = li.select('span[class="listing-street-address"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(streetAddress)
        
        # Collects city
        city = li.select('span[class="listing-city"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(city)
         
        # Collects state (region for the possibility of international use)
        region = li.select('span[class="listing-region"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(region)
        
        # Collects postal code
        postal = li.select('span[class="listing-postal"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(postal)
        
        # Collects price displayed. Could be lower bound, range, etc.
        if bool(li.select('span[class="data-price"]')) == True:
            dataPrice = li.select('span[class="data-price"]')[0].text.strip(' \t\n\r')
        else:
            dataPrice = na
        if isDebug:
            print("dataPrice is empty")
        
        if bool(li.select('span[class="price-addon-text"]')) == True:
            priceAddonText = li.select('span[class="price-addon-text"]')[0].text.strip(' \t\n\r')
        else:
            priceAddonText = na
        if isDebug:
            print("priceAddonText is empty")
            
        # Collects longitude and latitude
        lat = ''
        lon = ''
        
        if bool(li.select('span[class="listing-geo"]')) == True:
            for tag in li.select('span[class="listing-geo"]')[0].find_all("meta"):
                if tag.get("itemprop", None) == "latitude":
                    lat = tag.get("content", None)
                if tag.get("itemprop", None) == "longitude":
                    lon = tag.get("content", None)
                    
        # Collects property type
        if bool(li.select('div[class="property-type"]')) == True:
            propType = li.select('div[class="property-type"]')[0].text.strip(' \t\n\r')
            propType = propType.replace(' ','')
            propType = propType.replace('forRent','')
        else:
            propType = na
        if isDebug:
            print("propType is empty")
        
        # Collects bedrooms
        if bool(li.select('li[data-label="property-meta-beds"]')) == True:    
            bed = li.select('li[data-label="property-meta-beds"]')[0].text.strip(' ,\t\n\r')
            bed = bed.replace(' ','')
            bed = bed.replace('bed','')
        else:
            bed = na
        if isDebug:
            print("bed is empty")
            
        # Collects bathrooms
        if bool(li.select('li[data-label="property-meta-baths"]')) == True:
            bath = li.select('li[data-label="property-meta-baths"]')[0].text.strip(' ,\t\n\r')
            bath = bath.replace(' ','')
            bath = bath.replace('bath','')
        else:
            bath = na
        if isDebug:
            print("bath is empty")
            
       # Collects sqft
        if bool(li.select('li[data-label="property-meta-sqft"]')) == True:
            sqft = li.select('li[data-label="property-meta-sqft"]')[0].text.strip(' ,\t\n\r')
            sqft = sqft.replace(' ','')
            sqft = sqft.replace('sqft','')
        else:
            sqft = na
        if isDebug:
            print(sqft)
            
        source = "Realtor.com"
        
        mode = "rent"
     
        # Determine if property has Dayton address (based on city field)
        if(city.upper().find("DAYTON") >= 0):
            addressPriceList.append([streetAddress, city, region, postal, lat, lon, dataPrice, priceAddonText, propType, bed, bath, sqft, mode, source])
        else:
            if isDebug:
                print('out city: ', [streetAddress, city, region, postal, lat, lon, dataPrice, priceAddonText, propType, bed, bath, sqft, mode, source])
            continue

    # Print number of properties on page. Should be used as a check to make sure scraper is passing through all properties.
    print('# cards in page: ', i)
    
    # Print number of properties listed in Dayton, OH. These properties will be written to csv file.
    print('# cards in city: ', len(addressPriceList))
    
    return addressPriceList
    
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(i))

In [5]:
#Isolates and call information for each individual property
def get_property_card(city, state, page=1, isDebug=True):
    
    url = 'https://www.realtor.com/apartments/' + city + '_' + state + '/radius-1/pg-' + str(page)
    
    '''
    Use this url to make debugging easier. This way, the scraper does not cycle through every single page.
    
    url='https://www.realtor.com/realestateandhomes-search/Dayton_OH/radius-1/pg-1'
    '''
    
    print(url)
    
    '''
    Use this code for debugging. Code returns html file of page and stops. Use it isolate pieces of information and ensure return is html.
    
    response = simple_get(url)
    html = BeautifulSoup(response, 'html.parser')
    return html
    '''

    response = simple_get(url)

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        if isDebug:
            print(html)
            
        return get_address_price(html, isDebug)
    
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))

In [6]:
# Test cell. Ensure scraper is running properly by testing with first page.
property_cards = get_property_card('Dayton', 'OH', 1, isDebug=True)

https://www.realtor.com/apartments/Dayton_OH/radius-1/pg-1
<!DOCTYPE html>

<html lang="en">
<head>
<link href="https://static.rdc-next-basecamp.rdc.moveaws.com/assets/favicon-144287b492113455703667bc3182768a9b2666765a1cd4beba38cfe40294eccc.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=com.move.realtor" name="google-play-app"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5, viewport-fit=cover" name="viewport"/>
<meta content="It is now very easy to locate apartments for rent in Dayton, OH with the help of realtor.com&amp;reg;. Find 360 Dayton apartments and rentals now." name="description">
<meta content="authenticity_token" name="csrf-param">
<meta content="t1LNvat09/F6kmNRDRuw+ZUTSyKHeXaYQzCHvbvGdB1GVJw0gFfVG376M1YDbcIwWUdYLC2XOTKAgJC9IbYJlw==" name="csrf-token"/>
<link data-turbolinks-track="true" href="https://static.rdc-next-basecamp.rdc.moveaws.com/assets/application-26571c67d60ff3

priceAddonText is empty
propType is empty
bed is empty
bath is empty
637+
3839 Cloud Park Dr
Dayton
OH
45424
dataPrice is empty
priceAddonText is empty
propType is empty
bed is empty
bath is empty
775+
4026 Promenade Blvd
Beavercreek
OH
45431
dataPrice is empty
priceAddonText is empty
propType is empty
bed is empty
bath is empty
763+
out city:  ['4026 Promenade Blvd', 'Beavercreek', 'OH', '45431', '39.76947', '-84.082531', '$935+', '/month', 'Apartment', '1-2', '1-2', '763+', 'rent', 'Realtor.com']
350 Arden Way
Dayton
OH
45459
dataPrice is empty
priceAddonText is empty
propType is empty
bed is empty
bath is empty
787+
1661 Beaver Ridge Dr
Dayton
OH
45429
dataPrice is empty
priceAddonText is empty
propType is empty
bed is empty
bath is empty
710+
4135 Brookdale Ln
Dayton
OH
45440
dataPrice is empty
priceAddonText is empty
propType is empty
bed is empty
bath is empty
689+
9600 Summit Point Dr
Miamisburg
OH
45342
dataPrice is empty
priceAddonText is empty
propType is empty
bed is empty
b

bed is empty
bath is empty
569+
out city:  ['8310 Lyons Gate Way', 'Miamisburg', 'OH', '45342', '39.628722', '-84.233622', '$515+', '/month', 'Apartment', '1-3', '1-2', '569+', 'rent', 'Realtor.com']
# cards in page:  44
# cards in city:  16


In [7]:
# Test cell. Ensure scraper is only returning properties listed in Dayton, OH.
print(property_cards)

[['531 Belmonte Park N', 'Dayton', 'OH', '45405', '39.76727', '-84.20409', '$865', '/month', 'Apartment', '1', '1', '855+', 'rent', 'Realtor.com'], ['5010 Darby Rd', 'Dayton', 'OH', '45431', '39.78046', '-84.12307', '$535+', '/month', 'Apartment', '0-4', '1-2', '486+', 'rent', 'Realtor.com'], ['4001 Indian Runn Dr', 'Dayton', 'OH', '45415', '39.816352', '-84.237655', '$540+', '/month', 'Apartment', '1-2', '1-2', '637+', 'rent', 'Realtor.com'], ['3839 Cloud Park Dr', 'Dayton', 'OH', '45424', '39.81927', '-84.123726', '$720+', '/month', 'Apartment', '1-2', '1-2', '775+', 'rent', 'Realtor.com'], ['350 Arden Way', 'Dayton', 'OH', '45459', '39.64699', '-84.15051', '$1,180+', '/month', 'Apartment', '1-2', '1-2', '787+', 'rent', 'Realtor.com'], ['1661 Beaver Ridge Dr', 'Dayton', 'OH', '45429', '39.6982', '-84.13774', '$555+', '/month', 'Apartment', '1-3', '1-2', '710+', 'rent', 'Realtor.com'], ['4135 Brookdale Ln', 'Dayton', 'OH', '45440', '39.658743', '-84.092553', '$975+', '/month', 'Apartm

In [8]:
# Test cell. Count output of previous cell and check for match.
len(property_cards)

16

In [9]:
# Run scraper for all addresses in Dayton
totalAddressList = list()
i = 1
while True:
    addresses = get_property_card('Dayton', 'OH', i, isDebug=False)
    print(addresses)
    l = len(addresses)
    print(l)
    print(i)
    if l > 0 :
        totalAddressList = totalAddressList + addresses
        i = i + 1
    else:
        break
        
print(totalAddressList)
print(len(totalAddressList))

https://www.realtor.com/apartments/Dayton_OH/radius-1/pg-1
# cards in page:  44
# cards in city:  16
[['531 Belmonte Park N', 'Dayton', 'OH', '45405', '39.76727', '-84.20409', '$865', '/month', 'Apartment', '1', '1', '855+', 'rent', 'Realtor.com'], ['5010 Darby Rd', 'Dayton', 'OH', '45431', '39.78046', '-84.12307', '$535+', '/month', 'Apartment', '0-4', '1-2', '486+', 'rent', 'Realtor.com'], ['4001 Indian Runn Dr', 'Dayton', 'OH', '45415', '39.816352', '-84.237655', '$540+', '/month', 'Apartment', '1-2', '1-2', '637+', 'rent', 'Realtor.com'], ['3839 Cloud Park Dr', 'Dayton', 'OH', '45424', '39.81927', '-84.123726', '$720+', '/month', 'Apartment', '1-2', '1-2', '775+', 'rent', 'Realtor.com'], ['350 Arden Way', 'Dayton', 'OH', '45459', '39.64699', '-84.15051', '$1,180+', '/month', 'Apartment', '1-2', '1-2', '787+', 'rent', 'Realtor.com'], ['1661 Beaver Ridge Dr', 'Dayton', 'OH', '45429', '39.6982', '-84.13774', '$555+', '/month', 'Apartment', '1-3', '1-2', '710+', 'rent', 'Realtor.com'],

# cards in page:  44
# cards in city:  20
[['2581 Tidewater Dr', 'Dayton', 'OH', '45424', '39.80896', '-84.13133', '$590+', '/month', 'Apartment', '2-3', '1', 'n/a', 'rent', 'Realtor.com'], ['119 Edgewood Ave', 'Dayton', 'OH', '45402', '39.764048', '-84.209055', '$369+', '/month', 'Apartment', '1-2', '1', '710+', 'rent', 'Realtor.com'], ['743 Old Harshman Rd', 'Dayton', 'OH', '45431', '39.775644', '-84.123353', '$499+', '/month', 'Apartment', '1-2', '1', '576+', 'rent', 'Realtor.com'], ['15 Rue Royale', 'Dayton', 'OH', '45429', '39.70354', '-84.17402', 'n/a', 'n/a', 'Apartment', '2', '1-2', '1,440+', 'rent', 'Realtor.com'], ['805 Wilmington Ave', 'Dayton', 'OH', '45420', '39.73042', '-84.15792', '$550+', '/month', 'Apartment', '1-2', '1', '500+', 'rent', 'Realtor.com'], ['1017 Wilmington Ave', 'Dayton', 'OH', '45420', '39.72728', '-84.15583', '$485+', '/month', 'Apartment', '1-2', '1', '625+', 'rent', 'Realtor.com'], ['101 Fairground Ave Apt C', 'Dayton', 'OH', '45409', '39.74262', '-8

# cards in page:  44
# cards in city:  27
[['4459 Catalina Ave', 'Dayton', 'OH', '45416', '39.801312', '-84.263739', '$695', '/month', 'House', '3', '1', '864', 'rent', 'Realtor.com'], ['1948 Victoria Ave', 'Dayton', 'OH', '45406', '39.784143', '-84.235754', '$500', '/month', 'Apartment', '1', '1', '580+', 'rent', 'Realtor.com'], ['77 Baltimore St', 'Dayton', 'OH', '45404', '39.77749', '-84.171201', '$745', '/month', 'House', '3', '1', '1,319', 'rent', 'Realtor.com'], ['1501 Glenbeck Ave', 'Dayton', 'OH', '45409', '39.716041', '-84.196223', '$1,425', '/month', 'House', '3', '1.5', '1,640', 'rent', 'Realtor.com'], ['2517 California Ave', 'Dayton', 'OH', '45419', '39.710915', '-84.155388', '$1,225', '/month', 'House', '3', '2', '1,398', 'rent', 'Realtor.com'], ['1944 Sunny Ridge Rd N', 'Dayton', 'OH', '45414', '39.843583', '-84.199055', '$1,025', '/month', 'House', '4', '1', '1,008', 'rent', 'Realtor.com'], ['1317 Salem Ave', 'Dayton', 'OH', '45406', '39.776987', '-84.218161', '$500', '/

# cards in page:  8
# cards in city:  5
[['29 Laura Ave', 'Dayton', 'OH', '45405', '39.788015', '-84.21151', '$650', '/month', 'Apartment', '1', '1', 'n/a', 'rent', 'Realtor.com'], ['3097 Silver Rock Ave', 'Dayton', 'OH', '45414', '39.844977', '-84.206361', '$795+', '/month', 'Apartment', '2-3', '2-3', 'n/a', 'rent', 'Realtor.com'], ['500 Oak St', 'Dayton', 'OH', '45410', '39.748012', '-84.177983', '$850', '/month', 'Apartment', '3', '2', 'n/a', 'rent', 'Realtor.com'], ['2991 Benchwood Rd', 'Dayton', 'OH', '45414', '39.844016', '-84.206161', '$825', '/month', 'Apartment', '3', '2', 'n/a', 'rent', 'Realtor.com'], ['475 W Grand Ave', 'Dayton', 'OH', '45405', '39.769485', '-84.205758', '$500+', '/month', 'Apartment', '0-1', '1', '500+', 'rent', 'Realtor.com']]
5
9
https://www.realtor.com/apartments/Dayton_OH/radius-1/pg-10
# cards in page:  0
# cards in city:  0
[]
0
10
[['531 Belmonte Park N', 'Dayton', 'OH', '45405', '39.76727', '-84.20409', '$865', '/month', 'Apartment', '1', '1', '855

In [10]:
# Test cell. Double-check that totalAddressList is in form that can be written to csv file. Should see "list" in output.
type(totalAddressList)

list

In [11]:
# Create csv file. Headers can be modified to refer to actual values in list.
import csv

header = ['streetAddress', 'city', 'region', 'postal', 'lat', 'lon', 'dataPrice', 'priceAddonText', 'propType', 'bed', 'bath', 'sqft', 'mode', 'source']
type(header)

list

In [12]:
# Write totalAddressList to csv file and download csv file.
with open('realtorRent.csv', 'w') as f:
    csv_writer = csv.writer(f)
 
    csv_writer.writerow(header) # write header
 
    for row in totalAddressList:
        csv_writer.writerow(row)
        
print("Done")

Done
